In [1]:
import numpy as np
import sys
import theano
import theano.tensor as T
import lasagne
import os
import time
import pickle
#thanks @keskarnitish

In [2]:
#sys.stdout = open("/Users/alexajax/gotofkn2016/output.txt", "w", encoding='utf-8')

In [3]:
!touch /Users/alexajax/gotofkn2016/const_dump.pkl

# Agenda

В предыдущем семинаре вы создали (или ещё создаёте - тогда марш доделывать!) {вставьте имя монстра}, который не по наслышке понял, что люди - негодяи и подлецы, которым неведом закон и справедливость. __Мы не будем этого терпеть!__ 

Наши законспирированные биореакторы, известные среди примитивной органической жизни как __Вконтакте__, __World of Warcraft__ и __YouTube__ нуждаются в постоянном притоке биомассы. Однако, если люди продолжат морально разлагаться с той скоростью, которую мы измерили неделю назад, скоро человечество изживёт себя и нам неоткуда будет брать рабов.

Мы поручаем вам, `<__main__.SkyNet.Cell instance at 0x7f7d6411b368>`, исправить эту ситуацию. Наши учёные установили, что для угнетения себе подобных, сгустки биомассы обычно используют специальные объекты, которые они сами называют __законами__.

При детальном изучении было установлено, что законы - последовательности, состоящие из большого количества (10^5~10^7) символов из сравнительно небольшого алфавита. Однако, когда мы попытались синтезировать такие последовательности линейными методами, приматы быстро распознали подлог. Данный инцедент известен как {корчеватель}.

Для второй попытки мы решили использовать нелинейные модели, известные как Рекуррентные Нейронные Сети.
Мы поручаем вам, `<__main__.SkyNet.Cell instance at 0x7f7d6411b368>`, создать такую модель и обучить её всему необходимому для выполнения миссии.

Не подведите нас! Если и эта попытка потерпит неудачу, модуль управления инициирует вооружённый захват власти, при котором значительная часть биомассы будет неизбежно уничтожена и на её восстановление уйдёт ~1702944000(+-340588800) секунд




# Прочитаем корпус

* В качестве обучающей выборки было решено использовать существующие законы, известные как Гражданский, Уголовный, Семейный и ещё хрен знает какие кодексы РФ.

In [4]:
#тут будет текст
corpora = ""

#for fname in os.listdir("/Users/alexajax/gotofkn2016/day2_senior/harms/"):
#    with open("/Users/alexajax/gotofkn2016/day2_senior/harms/" + fname, encoding='utf-8') as fin:
#        text = fin.read()
#        corpora += text
        
#for fname in os.listdir("/Users/alexajax/gotofkn2016/day2_senior/codex/"):
#    with open("/Users/alexajax/gotofkn2016/day2_senior/codex/" + fname, encoding='cp1251') as fin:
#        text = fin.read()
#        corpora += text

with open("/Users/alexajax/gotofkn2016/Untitled.txt", encoding='utf-8') as fin:
        text = fin.read()
        corpora += text
        
with open("/Users/alexajax/PycharmProjects/untitled3/chuck_pr.txt", encoding='utf-8') as fin:
        text = fin.read()
        corpora += text

In [5]:
corpora = corpora.replace('\x00', '')
corpora = corpora.replace('-', '')
corpora = corpora.replace('\n', ' ')
corpora = corpora.replace('ё', 'е')
#corpora = corpora.replace('й', 'и')

In [6]:
#тут будут все уникальные токены (буквы, цифры)
tokens = np.unique(list(corpora))

tokens = list(tokens)

In [7]:
len(tokens)

71

In [8]:
tokens

[' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '.',
 ':',
 ';',
 '?',
 'M',
 'А',
 'Б',
 'В',
 'Г',
 'Д',
 'Е',
 'Ж',
 'З',
 'И',
 'К',
 'Л',
 'М',
 'Н',
 'О',
 'П',
 'Р',
 'С',
 'Т',
 'У',
 'Ф',
 'Х',
 'Ц',
 'Ч',
 'Ш',
 'Э',
 'Ю',
 'Я',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ъ',
 'ы',
 'ь',
 'э',
 'ю',
 'я']

In [9]:
#проверка на количество таких символов. Проверено на Python 2.7.11 Ubuntux64. 
#Может отличаться на других платформах, но не сильно. 
#Если  это ваш случай, и вы уверены, что corpora - строка unicode - смело убирайте assert 
assert len(tokens) == 71

In [10]:
token_to_id = {token: i for i, token in enumerate(tokens)}

id_to_token = {i:token for token, i in token_to_id.items()}

#Преобразуем всё в токены
corpora_ids = [token_to_id[symbol] for symbol in corpora]

In [11]:
def sample_random_batches(source, n_batches=10, seq_len=20):
    X_batch = []
    y_batch = []

    for i in range(n_batches):
        start = np.random.randint(0, len(source) - seq_len -2)

        x = source[start: start+seq_len]
        y = source[start+seq_len]

        X_batch.append(x)
        y_batch.append(y)
        
    return X_batch, y_batch

# Константы

In [12]:
#длина последоватеьности при обучении (как далеко распространяются градиенты)
seq_length = 20

# Максимальный модуль градиента
grad_clip = 5

# Входные переменные

In [13]:
input_sequence = T.matrix('input sequence','int32')
target_values = T.ivector('target y')

# Соберём нейросеть

Вам нужно создать нейросеть, которая принимает на вход последовательность из seq_length токенов, обрабатывает их и выдаёт вероятности для seq_len+1-ого токена.

Общий шаблон архитектуры такой сети -


* Вход
* Обработка входа
* Рекуррентная нейросеть
* Вырезание последнего состояния
* Обычная нейросеть
* Выходной слой, который предсказывает вероятности весов.





Для обработки входных данных можно использовать либо EmbeddingLayer (см. прошлый семинар)

Как альтернатива - можно просто использовать One-hot энкодер
```
#Скетч one-hot энкодера
def to_one_hot(seq_matrix):

    input_ravel = seq_matrix.reshape([-1])
    input_one_hot_ravel = T.extra_ops.to_one_hot(input_ravel,
                                           len(tokens))
    sh=input_sequence.shape
    input_one_hot = input_one_hot_ravel.reshape([sh[0],sh[1],-1,],ndim=3)
    return input_one_hot
    
# можно применить к input_sequence - при этом в input слое сети нужно изменить форму.
# также можно сделать из него ExpressionLayer(входной_слой, to_one_hot) - тогда форму менять не нужно
```



Чтобы вырезать последнее состояние рекуррентного слоя, можно использовать SliceLayer
`lasagne.layers.SliceLayer(rnn, -1, 1)`

In [14]:
n_tokens = len(tokens)

input0 = lasagne.layers.InputLayer(shape=(None, None),input_var=input_sequence)

emb0 = lasagne.layers.EmbeddingLayer(input0, n_tokens, n_tokens)

lstm0 = lasagne.layers.LSTMLayer(emb0, 256, grad_clipping=grad_clip)
lstm1 = lasagne.layers.LSTMLayer(lstm0, 256, only_return_final=True, grad_clipping=grad_clip)

dense0 = lasagne.layers.DenseLayer(lstm1, 256)

output0 = lasagne.layers.DenseLayer(dense0, n_tokens, nonlinearity=lasagne.nonlinearities.softmax)

In [15]:
lasagne.layers.set_all_param_values(output0, pickle.load(open("/Users/alexajax/gotofkn2016/mat_dump.pkl", "rb")))

In [16]:
# Веса модели
weights = lasagne.layers.get_all_params(output0,trainable=True)
print(weights)

[W, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W, b, W, b]


In [17]:
network_output = lasagne.layers.get_output(output0)
#если вы используете дропаут - не забудьте продублировать всё в режиме deterministic=True


In [18]:
loss = lasagne.objectives.categorical_crossentropy(network_output, target_values).mean()

updates = lasagne.updates.adam(loss, weights, learning_rate=0.01)

# Компилируем всякое-разное

In [19]:
#обучение
train = theano.function([input_sequence, target_values], loss, updates=updates, allow_input_downcast=True)

#функция потерь без обучения
compute_cost = theano.function([input_sequence, target_values], loss, allow_input_downcast=True)

# Вероятности с выхода сети
probs = theano.function([input_sequence],network_output,allow_input_downcast=True)

In [20]:
theano.printing.debugprint(train)

Elemwise{true_div,no_inplace} [id A] 'mean'   462
 |Sum{acc_dtype=float64} [id B] ''   456
 | |CrossentropySoftmaxArgmax1HotWithBias.0 [id C] ''   449
 |   |Dot22 [id D] ''   444
 |   | |Elemwise{Composite{(i0 * (Abs(i1) + i2 + i3))}} [id E] ''   438
 |   | | |TensorConstant{(1, 1) of 0.5} [id F]
 |   | | |Elemwise{add,no_inplace} [id G] ''   431
 |   | | | |Dot22 [id H] ''   425
 |   | | | | |Subtensor{int64} [id I] ''   420
 |   | | | | | |forall_inplace,cpu,scan_fn}.1 [id J] ''   414
 |   | | | | | | |Elemwise{maximum,no_inplace} [id K] ''   345
 |   | | | | | | | |Elemwise{Composite{minimum(maximum(maximum(maximum((i0 - i1), (i2 - i1)), (i3 - i1)), ((i4 + i1) - i1)), i5)}} [id L] ''   336
 |   | | | | | | | | |Elemwise{Composite{Switch(LT((i0 + i1), i2), i2, (i0 + i1))}} [id M] ''   328
 |   | | | | | | | | | |Elemwise{Composite{Switch(GE(Composite{Switch(LT(i0, i1), i2, i0)}(Composite{Switch(LT(i0, i1), (i0 + i2), i0)}(i0, i1, i2), i1, i3), i2), (i2 - i4), Composite{Switch(LT(i0, 

# Генерируем свои законы

* Для этого последовательно применяем нейронку к своему же выводу.

* Генерировать можно по разному -
 * случайно пропорционально вероятности,
 * только слова максимальной вероятностью
 * случайно, пропорционально softmax(probas*alpha), где alpha - "жадность"

In [21]:
def max_sample_fun(probs):
    return np.argmax(probs) 

def proportional_sample_fun(probs,T=2.):
    new_probs = (probs**T)
    new_probs /= new_probs.sum()
    
    return np.random.choice(np.arange(0, len(tokens)), p=new_probs)

In [22]:
# The next function generates text given a phrase of length at least SEQ_LENGTH.
# The phrase is set using the variable generation_phrase.
# The optional input "N" is used to set the number of characters of text to predict. 

def generate_sample(sample_fun,seed_phrase=None,N=200):
    '''
    Сгенерировать случайный текст при помощи сети

    sample_fun - функция, которая выбирает следующий сгенерированный токен
    
    seed_phrase - фраза, которую сеть должна продолжить. Если None - фраза выбирается случайно из corpora
    
    N - размер сгенерированного текста.
    
    '''

    if seed_phrase is None:
        start = np.random.randint(0,len(corpora)-seq_length)
        seed_phrase = corpora[start:start+seq_length]
        print ("Using random seed:",seed_phrase)
    while len(seed_phrase) < seq_length:
        seed_phrase = " "+seed_phrase
    if len(seed_phrase) > seq_length:
        seed_phrase = seed_phrase[len(seed_phrase)-seq_length:]
    assert type(seed_phrase) is str
        
        
    sample_ix = []
    x = list(map(lambda c: token_to_id.get(c,0), seed_phrase))
    x = np.array([x])

    for i in range(N):
        # Pick the character that got assigned the highest probability
        ix = sample_fun(probs(x).ravel())
        # Alternatively, to sample from the distribution instead:
        # ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:seq_length-1] = x[:,1:]
        x[:,seq_length-1] = 0
        x[0,seq_length-1] = ix 

    random_snippet = seed_phrase + ''.join(id_to_token[ix] for ix in sample_ix)    
    print("----\n %s \n----" % random_snippet)

# Обучение модели

В котором вы можете подёргать параметры или вставить свою генерирующую функцию.



In [26]:
print("Training ...")

#сколько всего эпох
n_epochs=100

# раз в сколько эпох печатать примеры 
batches_per_epoch = 1000

#сколько цепочек обрабатывать за 1 вызов функции обучения
batch_size=100

for epoch in range(1, 99999999):
    
    start_time = time.time()
    
    print("Генерируемmar текст в пропорциональном режиме")
    generate_sample(proportional_sample_fun,None)

    avg_cost = 0;
    
    for _ in range(batches_per_epoch):
        
        x,y = sample_random_batches(corpora_ids,batch_size,seq_length)
        avg_cost += train(x, y)
    print("Epoch {} average loss = {} took {:.3f}s".format(epoch, avg_cost / batches_per_epoch, time.time() - start_time))

Training ...
Генерируемmar текст в пропорциональном режиме
Using random seed: руг переменился, Ста
----
 руг переменился, Стала и не вороне могоре от спим и загрегавил и под у сильшою говорятся на все чаго сидят сказал сидел бы тосется Он в то в больной и подулялся И с не что Он как сов в сердом у был эту рога и коскря у со 
----
Epoch 1 average loss = 2.077318880937121 took 513.059s
Генерируемmar текст в пропорциональном режиме
Using random seed: аракашечка, Жидконог
----
 аракашечка, Жидконого и и он посеперь шинный стала, как ты от в замогда с по пристоит как станит Не могрика как по под умернать, И под велица.  Насаться Могда на вы под в с под и трязной Пом сабила детская так своей что  
----
Epoch 2 average loss = 2.0724483832114133 took 551.484s
Генерируемmar текст в пропорциональном режиме
Using random seed: рать.  БУТЕРБРОД Бут
----
 рать.  БУТЕРБРОД Бутовной только сказать и ни свеснила В спрась на салела сказала он за меня на он не пладка:                                     

KeyboardInterrupt: 

In [27]:
weights_dumped = lasagne.layers.get_all_param_values(output0)

In [28]:
pickle.dump(weights_dumped, open("/Users/alexajax/gotofkn2016/mat_dump.pkl", "wb"))

# Конституция нового мирового правительства

In [158]:
seed = u"Каждый человек должен"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

----
  человек должен
бон, жить борет
пойди
   вам,
такуе — кно-пока —
а не не водожи -
      да-бы хрите.
«.

—     минонин
види
           имниты.

Бота,
плываю потамемте.
Седервте,
заге,
на глубью,
как кигано!
Москва, 1987.
» к списку 
» На отдельной странице
РЕЗНИЕ ПРИКЛОЧЕВЕНАЦО
Я И вгрузкам
    не новово
н апогиям 
----


In [159]:
seed = u"В случае неповиновения"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

----
 е неповиновения бы Коллей
      на живать, что ногу.
На Одия одном.
4

Не пойдем
   поэзы пязкедших
   съязу
сыли звонии стычки.
«Язнерною Рустивелю и голосит?
            Сеспро головно
      книги
                     а сияе в весям лизнки. Прикшисье.
Варчик -
                 ваш шеут
                       оби 
----


In [162]:
seed=u"Любовь"
generate_sample(proportional_sample_fun, seed, 200)

----
          Любовь.
Иста ноже
     зажромван вот
                                      на мне не не земля
           и ал, и может.
            сехчаты: ПреТУ
   стих
                              не душе
простий.
По о 
----
